In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 26 16:53:02 2023

@author: gfoti
"""

from agents import *


class Food(Thing):
    pass

class Water(Thing):
    pass


In [2]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        agent.display()
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        # Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    #in agent create functions instead of in dog class.
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
            agent.stepped()
        elif action == 'down':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location)) 
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    agent.scored('ate')
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    agent.scored('drank')
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles



In [3]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    steps=0
    score=0
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1

    def stepped(self):
        self.steps+=1
    def scored(self, action):
        if(action == 'ate'):
            self.score+=25
        if(action == 'drank'):
            self.score+=10
    def display(self):
        print("Steps taken:", self.steps)
        if self.score == 105:
            print("You completed the game!")
        if self.score == 30:
            print("You drank all the water!")
        if self.score == 75:
            print("You ate all the food!")
        print("Score:", self.score)
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

import keyboard  

def program(percepts):
    '''Returns an action based on it's percepts'''

    for p in percepts: 
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        if isinstance(p,Bump): # if at an edge and have to turn
            turn = False
            choice = random.choice((1,2)); #will have to change this to stay at the same square
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    while True:    
        
        key=keyboard.read_key();
        if key == 'd':
            return 'turnright'
        elif key == 'a':
            return 'turnleft'
        elif key == 'w':
            return 'moveforward'
         #elif keyboard.read_key() == 's'
            #return 'movebackward'
        else:
            continue
    

In [4]:
#version 1 - doesn't work, rethink
import random
def randcoors(num):
    
    #num is the number of random coordinates
    #need coordinates between ([1,8],[1,8]) greater than or equal to 1 and less than 8. Highest index 7
    coordlist = []
    for i in range(num):
        while x not in list and y not in list:
            x = random.choice([1,2,3,4,5,6,7])
            y = random.choice([1,2,3,4,5,6,7])
            coordlist = coordlist+[x,y]
        print(coordlist)
        

In [5]:
import random

def randcoor(num):
    #an empty list to store the coordinates
    coordlist = []
    
    #valid range for coordinates
    valid_range = list(range(1, 8))
    
    while len(coordlist) < 2 * num:  #Repeat until we have enough unique coordinates
        x = random.choice(valid_range)
        y = random.choice(valid_range)
        
        #Check if the generated coordinates are not already in coordlist
        if (x, y) not in zip(coordlist[::2], coordlist[1::2]):
            #Append the coordinates
            coordlist.extend([x, y])
    
    
    return coordlist



In [6]:
park = Park2D(8,8, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})
dog = EnergeticBlindDog(program)
dogfood1 = Food()
dogfood2 = Food()
dogfood3 = Food()
water1 = Water()
water2 = Water()
water3 = Water()
coordlist = randcoor(6)

park.add_thing(dogfood1, [coordlist[0],coordlist[1]])
park.add_thing(dogfood2, [coordlist[2],coordlist[3]])
park.add_thing(dogfood3, [coordlist[4],coordlist[5]])
park.add_thing(water1, [coordlist[6],coordlist[7]])
park.add_thing(water2, [coordlist[8],coordlist[9]])
park.add_thing(water3, [coordlist[10],coordlist[11]])

park.add_thing(dog, [0,0])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.run(50)
print("Game over!")
dog.display()  


,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,


Game over!
Steps taken: 17
You completed the game!
Score: 105
